In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math
import catboost
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv('kaggle_4.csv')
print (len(df))
df.type.value_counts()

651191


type
benign        428103
defacement     96457
phishing       94111
malware        32520
Name: count, dtype: int64

In [6]:
Label = LabelEncoder()
df['type'] = Label.fit_transform(df['type'])
print (df.type.value_counts())
df.head()

type
0    428103
1     96457
3     94111
2     32520
Name: count, dtype: int64


,url,type
0,br-icloud.com.br,3
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,0
6,espn.go.com/nba/player/_/id/3457/brandon-rush,0
7,yourbittorrent.com/?q=anthony-hamilton-soulife,0
8,http://www.pashminaonline.com/pure-pashminas,1
9,allmusic.com/album/crazy-from-the-heat-r16990,0


Feature Extraction:

leangth, number of (., /, //, #), presence of unicode and https

In [7]:
df['url_length'] = df['url'].apply(len)
df['num_dots'] = df['url'].apply(lambda x: x.count('.'))
df['num_slash'] = df['url'].apply(lambda x: x.count('/'))
df['num_redir'] = df['url'].apply(lambda x: x.count('//'))
df['num_dash'] = df['url'].apply(lambda x: x.count('-'))
df['contains_anchor'] = df['url'].str.contains('#')
df['has_https'] = df['url'].str.contains("https")
df.head()

,url,type,url_length,num_dots,num_slash,num_redir,num_dash,contains_anchor,has_https
0,br-icloud.com.br,3,16,2,0,0,1,False,False
1,mp3raid.com/music/krizz_kaliko.html,0,35,2,2,0,0,False,False
2,bopsecrets.org/rexroth/cr/1.htm,0,31,2,3,0,0,False,False
3,http://www.garage-pirenne.be/index.php?option=...,1,88,3,3,1,1,False,False
4,http://adventure-nicaragua.net/index.php?optio...,1,235,2,3,1,1,False,False


In [9]:
def contains_unicode(url):
    for char in url:
        if ord(char) > 127:
            return True
    return False

df['contains_unicode'] = df['url'].apply(contains_unicode)
df.head()

,url,type,url_length,num_dots,num_slash,num_redir,num_dash,contains_anchor,has_https,contains_unicode
0,br-icloud.com.br,3,16,2,0,0,1,False,False,False
1,mp3raid.com/music/krizz_kaliko.html,0,35,2,2,0,0,False,False,False
2,bopsecrets.org/rexroth/cr/1.htm,0,31,2,3,0,0,False,False,False
3,http://www.garage-pirenne.be/index.php?option=...,1,88,3,3,1,1,False,False,False
4,http://adventure-nicaragua.net/index.php?optio...,1,235,2,3,1,1,False,False,False


In [11]:
input = df.drop(['url', 'type'], axis='columns')
target = df.type
print(target.head())
input.head()

0    3
1    0
2    0
3    1
4    1
Name: type, dtype: int64


,url_length,num_dots,num_slash,num_redir,num_dash,contains_anchor,has_https,contains_unicode
0,16,2,0,0,1,False,False,False
1,35,2,2,0,0,False,False,False
2,31,2,3,0,0,False,False,False
3,88,3,3,1,1,False,False,False
4,235,2,3,1,1,False,False,False


In [12]:
len (input)


651191

In [13]:
X_train, X_test, y_train, y_test = train_test_split(input, target, test_size=0.2)
print (len (X_test))
print (len (X_train))

130239
520952


Random Forest

In [21]:
model = RandomForestClassifier(n_estimators=30)
model.fit (X_train, y_train)

RandomForestClassifier(n_estimators=30)

In [22]:
y_pred = model.predict (X_test)
print (classification_report (y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95     85808
           1       0.89      0.94      0.91     19329
           2       0.89      0.84      0.86      6461
           3       0.78      0.65      0.71     18641

    accuracy                           0.91    130239
   macro avg       0.87      0.85      0.86    130239
weighted avg       0.90      0.91      0.90    130239



Cat Boost

In [23]:
cat_features = []
cat_model = catboost.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, loss_function='MultiClass')
cat_model.fit (X_train, y_train, cat_features=cat_features)

0:	learn: 1.1976035	total: 252ms	remaining: 25s
1:	learn: 1.0641594	total: 338ms	remaining: 16.6s
2:	learn: 0.9599770	total: 429ms	remaining: 13.9s
3:	learn: 0.8785258	total: 520ms	remaining: 12.5s
4:	learn: 0.8123414	total: 625ms	remaining: 11.9s
5:	learn: 0.7576380	total: 733ms	remaining: 11.5s
6:	learn: 0.7101355	total: 829ms	remaining: 11s
7:	learn: 0.6701107	total: 924ms	remaining: 10.6s
8:	learn: 0.6346151	total: 1.02s	remaining: 10.3s
9:	learn: 0.6056169	total: 1.12s	remaining: 10s
10:	learn: 0.5791846	total: 1.23s	remaining: 9.96s
11:	learn: 0.5565808	total: 1.32s	remaining: 9.7s
12:	learn: 0.5363666	total: 1.41s	remaining: 9.41s
13:	learn: 0.5167382	total: 1.5s	remaining: 9.23s
14:	learn: 0.4990431	total: 1.59s	remaining: 9.01s
15:	learn: 0.4836211	total: 1.68s	remaining: 8.84s
16:	learn: 0.4692427	total: 1.78s	remaining: 8.68s
17:	learn: 0.4582173	total: 1.86s	remaining: 8.46s
18:	learn: 0.4474870	total: 1.94s	remaining: 8.28s
19:	learn: 0.4372609	total: 2.03s	remaining: 8.11

In [24]:
cat_pred = cat_model.predict(X_test)

In [25]:
report = classification_report(cat_pred, y_test)
confusion = confusion_matrix(cat_pred, y_test)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", confusion)

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.92      0.94     89675
           1       0.92      0.84      0.88     21246
           2       0.70      0.84      0.77      5359
           3       0.58      0.78      0.67     13959

    accuracy                           0.89    130239
   macro avg       0.79      0.85      0.81    130239
weighted avg       0.91      0.89      0.90    130239

Confusion Matrix:
 [[82852   681   251  5891]
 [  591 17863  1236  1556]
 [  107   401  4526   325]
 [ 2258   384   448 10869]]


SMOTE 

In [26]:
y_test.value_counts()

type
0    85808
1    19329
3    18641
2     6461
Name: count, dtype: int64

In [28]:
smote = SMOTE(sampling_strategy='minority', k_neighbors=4)

In [29]:
X_smt, y_smt = smote.fit_resample(X_train, y_train)

In [30]:
cat_model.fit (X_smt, y_smt, cat_features=cat_features)

0:	learn: 1.1990668	total: 171ms	remaining: 16.9s
1:	learn: 1.0679899	total: 313ms	remaining: 15.4s
2:	learn: 0.9693210	total: 467ms	remaining: 15.1s
3:	learn: 0.8911636	total: 607ms	remaining: 14.6s
4:	learn: 0.8260236	total: 750ms	remaining: 14.2s
5:	learn: 0.7711966	total: 894ms	remaining: 14s
6:	learn: 0.7257807	total: 1.04s	remaining: 13.8s
7:	learn: 0.6872775	total: 1.18s	remaining: 13.6s
8:	learn: 0.6531728	total: 1.32s	remaining: 13.3s
9:	learn: 0.6243661	total: 1.48s	remaining: 13.3s
10:	learn: 0.5979728	total: 1.63s	remaining: 13.2s
11:	learn: 0.5756548	total: 1.79s	remaining: 13.1s
12:	learn: 0.5559690	total: 1.95s	remaining: 13s
13:	learn: 0.5401300	total: 2.09s	remaining: 12.8s
14:	learn: 0.5243182	total: 2.24s	remaining: 12.7s
15:	learn: 0.5079817	total: 2.4s	remaining: 12.6s
16:	learn: 0.4936661	total: 2.57s	remaining: 12.5s
17:	learn: 0.4820937	total: 2.72s	remaining: 12.4s
18:	learn: 0.4701780	total: 2.86s	remaining: 12.2s
19:	learn: 0.4606756	total: 3s	remaining: 12s


In [31]:
cat_pred = cat_model.predict(X_test)

In [32]:
report = classification_report(cat_pred, y_test)
confusion = confusion_matrix(cat_pred, y_test)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", confusion)

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     88979
           1       0.70      0.91      0.79     14859
           2       0.92      0.41      0.57     14468
           3       0.51      0.80      0.62     11933

    accuracy                           0.85    130239
   macro avg       0.77      0.76      0.73    130239
weighted avg       0.88      0.85      0.85    130239

Confusion Matrix:
 [[82255   574   123  6027]
 [  486 13450   189   734]
 [ 1084  5095  5947  2342]
 [ 1983   210   202  9538]]
